In [29]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
import shap
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import shap

In [30]:
data = pd.read_csv('Cleaned_full_data.csv')

C:\Users\adi_s\AppData\Local\Temp\ipykernel_30888\2584280520.py:1: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Cleaned_full_data.csv')


In [31]:
# Reset retained index.
data = data.reset_index(drop=True)
# Set NA to 0.
data['ct_ftp_cmd'] = data['ct_ftp_cmd'].fillna(0)
data['attack_cat'] = data['attack_cat'].str.replace(r'\s+', '', regex=True)
data['attack_cat'] = data['attack_cat'].str.replace('Backdoors', 'Backdoor')

In [32]:
# Remove encoded columns.
data = data.drop(columns=['proto', 'dsport', 'service', 'state', 'srcip', 'sport', 'dstip'])
data['attack_cat'] = (data['attack_cat'] == 'DoS').astype(int)

In [33]:
features = data.drop(columns=['attack_cat', 'label'])
label = data['attack_cat']
attack_cat = data['attack_cat']
scaler = StandardScaler()
scaled_data = scaler.fit_transform(features)
scaled_data_df = pd.DataFrame(scaled_data, columns=features.columns)
scaled_data_df['attack_cat'] = attack_cat.reset_index(drop=True)
# Split 20% test.

In [34]:
# Updated thresholds to capture as much as possible whilst managing memory load - focusing on ports and protocols.
# See: TestCorrelationThresholds.ipynb in Archive folder.
ohe1 = pd.read_csv('DoS_proto_encoded.csv')
ohe2 = pd.read_csv('DoS_dsport_encoded.csv')
ohe3 = pd.read_csv('DoS_service_encoded.csv')
ohe4 = pd.read_csv('DoS_state_encoded.csv')
ohe5 = pd.read_csv('DoS_srcip_encoded.csv')
ohe6 = pd.read_csv('DoS_sport_encoded.csv')
ohe7 = pd.read_csv('DoS_dstip_encoded.csv')

In [35]:
# 338 features.
full_data = pd.concat([scaled_data_df, ohe1, ohe2, ohe3, ohe4, ohe5, ohe6, ohe7], axis=1)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(full_data, label, test_size=0.2, random_state=42)
# Drop attack cat before running model and store for later evaluation indexing.
test_attack_cat = X_test['attack_cat']
X_train = X_train.drop(columns=['attack_cat'])
X_test = X_test.drop(columns=['attack_cat'])
# Check label distribution.
print(scaled_data_df['attack_cat'].value_counts())

attack_cat
0    2523694
1      16353
Name: count, dtype: int64


In [38]:
print(X_test.shape)
print(X_train.shape)

(508010, 336)
(2032037, 336)


In [39]:
# Run RandomForest with default settings.
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

MemoryError: Unable to allocate 2.54 GiB for an array with shape (2032037, 336) and data type float32

In [ ]:
# Add labels back to the data and create a label list.
X_test['predicted_label'] = y_pred
X_test['true_label'] = y_test.values
X_test['attack_cat'] = test_attack_cat
X_test = X_test.reset_index(drop=True)
attack_categories = X_test['attack_cat'].unique()
# Iterate for each category label and group evaluation results and print.
for category in attack_categories:
    print(f"Evaluation for attack category: {category}")
    category_data = X_test[X_test['attack_cat'] == category]
    y_true_category = category_data['true_label']
    y_pred_category = category_data['predicted_label']
    correct_predictions = (y_true_category == y_pred_category).sum()
    total_samples = len(y_true_category)
    accuracy = accuracy_score(y_true_category, y_pred_category)
    precision = precision_score(y_true_category, y_pred_category, zero_division=0)
    recall = recall_score(y_true_category, y_pred_category, zero_division=0)
    f1 = f1_score(y_true_category, y_pred_category, zero_division=0)
    print(f"Total samples: {total_samples}")
    print(f"Correct predictions: {correct_predictions}")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")
    print("\nClassification Report:\n", classification_report(y_true_category, y_pred_category, zero_division=0))
    print("-" * 50)

Evaluation for attack category: 0
Total samples: 504695
Correct predictions: 504601
Accuracy: 0.9998137488978492
Precision: 0.0
Recall: 0.0
F1-Score: 0.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    504695
           1       0.00      0.00      0.00         0

    accuracy                           1.00    504695
   macro avg       0.50      0.50      0.50    504695
weighted avg       1.00      1.00      1.00    504695

--------------------------------------------------
Evaluation for attack category: 1
Total samples: 3315
Correct predictions: 104
Accuracy: 0.03137254901960784
Precision: 1.0
Recall: 0.03137254901960784
F1-Score: 0.060836501901140684

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.03      0.06      3315

    accuracy                           0.03      3315
   macro avg     

In [19]:
# Get correct and incorrect predictions. Create DataFrames then drop the columns.
X_test['incorrect_predictions'] = X_test['predicted_label'] != X_test['true_label']
X_test['correct_predictions'] = X_test['predicted_label'] == X_test['true_label']
incorrect_predictions = X_test[X_test['incorrect_predictions'] == True]
correct_predictions = X_test[X_test['correct_predictions'] == True]
sdata1 = incorrect_predictions.reset_index(drop=True).drop(columns=['correct_predictions', 'incorrect_predictions', 'predicted_label'])
sdata2 = correct_predictions.reset_index(drop=True).drop(columns=['correct_predictions', 'incorrect_predictions', 'predicted_label'])
# Validate.
print("Incorrect Predictions (All):")
print(len(sdata1))
print("Correct Predictions (All):")
print(len(sdata2))

Incorrect Predictions (All):
3305
Correct Predictions (All):
504705


In [20]:
sdata1 = sdata1.sample(n=1000)
sdata2 = sdata2.sample(n=1000)

In [21]:
sdata1 = sdata1.drop(columns=['true_label', 'attack_cat'])
sdata2 = sdata2.drop(columns=['true_label', 'attack_cat'])

In [22]:
explainer = shap.TreeExplainer(rf)
shap_values1 = explainer.shap_values(sdata1)

ExplainerError: Additivity check failed in TreeExplainer! Please ensure the data matrix you passed to the explainer is the same shape that the model was trained on. If your data shape is correct then please report this on GitHub. Consider retrying with the feature_perturbation='interventional' option. This check failed because for one of the samples the sum of the SHAP values was -10.657296, while the model output was 0.719062. If this difference is acceptable you can set check_additivity=False to disable this check.

In [ ]:
print(sdata1.shape)
print(shap_values1.shape)

(3305, 336)
(508010, 341)


In [ ]:
explainer = shap.TreeExplainer(rf)
shap_values2 = explainer.shap_values(sdata2)

# Can we improve accuracy in minority labels through SHAP feature engineering?

In [ ]:
print('Correct Predictions')
shap.summary_plot(shap_values1[:, :, 1], sdata1, max_display=len(100), show=True)
print('Incorrect Predictions')
shap.summary_plot(shap_values2[:, :, 1], sdata2, max_display=len(100), show=True)

In [ ]:
# Run RandomForest with default settings.
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))